In [1]:
pip install pandas numpy scikit-learn torch transformers tqdm


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm


In [3]:
# Load the dataset
# Replace 'your_dataset.csv' with your actual dataset path
df = pd.read_csv('/kaggle/input/twitter/train.csv')  # Ensure the file path is correct

# Display basic information
print(f"Total samples: {len(df)}")
#df=df[0:10000]
print(df.head())
print(df.shape)

Total samples: 1046343
   Y                                               text
0  1                                  kinder craft time
1  0                             i miss my seat partner
2  1                                             thanks
3  0                                     we alreay went
4  1  i don t think chorizo counts in any healthy ea...
(1046343, 2)


In [4]:
df=df.dropna()

In [5]:
print(df)

         Y                                               text
0        1                                  kinder craft time
1        0                             i miss my seat partner
2        1                                             thanks
3        0                                     we alreay went
4        1  i don t think chorizo counts in any healthy ea...
...     ..                                                ...
1046338  0          i don t like being locked out from things
1046339  1         still got that one packed it yesterday too
1046340  1  oh if only i had a garden i hardly ever miss h...
1046341  0  same shit lol sleepy but i need to charge my p...
1046342  1  dude you ve been quiet miss you can we hang ou...

[1046343 rows x 2 columns]


In [6]:
df['text'] = df['text'].fillna("").astype(str)
x, xt, y, yt = train_test_split(
    df['text'].values,
    df['Y'].values,
    test_size=0.9,
    random_state=15,
    stratify=df['Y'].values
)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    x,
    y,
    test_size=0.3,
    random_state=25,
    #stratify=df['y'].values
)


In [7]:
print(train_texts.size)

73243


In [8]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define maximum sequence length
MAX_LEN = 128

def tokenize_texts(texts, tokenizer, max_len):
    return tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

# Tokenize training and validation texts
train_encodings = tokenize_texts(train_texts, tokenizer, MAX_LEN)
val_encodings = tokenize_texts(val_texts, tokenizer, MAX_LEN)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
class SarcasmDataset(Dataset):
    def __init__(self, encodings, labels):
        self.input_ids = encodings['input_ids']
        self.attention_mask = encodings['attention_mask']
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Create Dataset objects
train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)


In [10]:
BATCH_SIZE = 16

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [11]:
class DoubleBERT(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', num_labels=2):
        super(DoubleBERT, self).__init__()
        # First BERT model for embeddings
        self.bert_encoder = BertModel.from_pretrained(bert_model_name)

        # Second BERT model for classification
        self.bert_classifier = BertModel.from_pretrained(bert_model_name)

        # Classification layer
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert_classifier.config.hidden_size * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        # Pass through the first BERT encoder
        encoder_outputs = self.bert_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        encoder_cls = encoder_outputs.last_hidden_state[:,0,:]  # CLS token

        # Pass through the second BERT classifier
        classifier_outputs = self.bert_classifier(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        classifier_cls = classifier_outputs.last_hidden_state[:,0,:]  # CLS token

        # Concatenate CLS tokens from both BERTs
        combined = torch.cat((encoder_cls, classifier_cls), dim=1)
        combined = self.dropout(combined)

        # Final classification layer
        logits = self.classifier(combined)

        return logits


In [12]:

# Check for GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Initialize the model
model = DoubleBERT()
model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

# Define number of training steps
epochs = 3
total_steps = len(train_loader) * epochs

# Define scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Define loss function
criterion = nn.CrossEntropyLoss()


Using device: cuda


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
def train_epoch(model, data_loader, optimizer, scheduler, device, criterion):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader, desc="Training"):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(data_loader)
    return avg_loss

def eval_model(model, data_loader, device, criterion):
    model.eval()
    total_loss = 0
    preds = []
    true_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            preds.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(true_labels, preds)
    report = classification_report(true_labels, preds, digits=4)
    return avg_loss, accuracy, report


In [14]:
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    print("-" * 20)

    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device, criterion)
    print(f"Training Loss: {train_loss:.4f}")

    val_loss, val_accuracy, val_report = eval_model(model, val_loader, device, criterion)
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print("Classification Report:")
    print(val_report)



Epoch 1/3
--------------------


Training: 100%|██████████| 4578/4578 [51:58<00:00,  1.47it/s]


Training Loss: 0.4058


Evaluating: 100%|██████████| 1962/1962 [07:20<00:00,  4.45it/s]


Validation Loss: 0.3747
Validation Accuracy: 0.8315
Classification Report:
              precision    recall  f1-score   support

           0     0.8051    0.8741    0.8381     15667
           1     0.8628    0.7891    0.8243     15724

    accuracy                         0.8315     31391
   macro avg     0.8339    0.8316    0.8312     31391
weighted avg     0.8340    0.8315    0.8312     31391


Epoch 2/3
--------------------


...

Validation Loss: 0.4231
Validation Accuracy: 0.8335
Classification Report:
              precision    recall  f1-score   support

           0     0.8291    0.8393    0.8342     15667
           1     0.8379    0.8276    0.8327     15724

    accuracy                         0.8335     31391
   macro avg     0.8335    0.8335    0.8335     31391
weighted avg     0.8335    0.8335    0.8335     31391


Epoch 3/3
--------------------


Training: 100%|██████████| 4578/4578 [52:04<00:00,  1.47it/s]


Training Loss: 0.1610


Evaluating: 100%|██████████| 1962/1962 [07:20<00:00,  4.46it/s]

Validation Loss: 0.6292
Validation Accuracy: 0.8289
Classification Report:
              precision    recall  f1-score   support

           0     0.8305    0.8258    0.8281     15667
           1     0.8274    0.8320    0.8297     15724

    accuracy                         0.8289     31391
   macro avg     0.8289    0.8289    0.8289     31391
weighted avg     0.8289    0.8289    0.8289     31391



In [ ]:
class_counts = pd.Series(y).value_counts().sort_index()  # Adjust if using df directly
total_samples = sum(class_counts)
class_weights = torch.tensor([total_samples / count for count in class_counts], dtype=torch.float32).to(device)

# Define weighted cross-entropy loss
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

# Define number of training steps
epochs = 3
total_steps = len(train_loader) * epochs

# Define scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

def train_epoch(model, data_loader, optimizer, scheduler, device, criterion):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader, desc="Training"):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(data_loader)
    return avg_loss

def eval_model(model, data_loader, device, criterion):
    model.eval()
    total_loss = 0
    preds = []
    true_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            preds.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(true_labels, preds)
    report = classification_report(true_labels, preds, digits=4)
    return avg_loss, accuracy, report

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    print("-" * 20)

    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device, criterion)
    print(f"Training Loss: {train_loss:.4f}")

    val_loss, val_accuracy, val_report = eval_model(model, val_loader, device, criterion)
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print("Classification Report:")
    print(val_report)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/3
--------------------


Training:   1%|▏         | 60/4578 [00:40<51:15,  1.47it/s]

In [ ]:
# Save the model
model_save_path = 'double_bert_sarcasm_detector.pt'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


In [ ]:
# To load the model later
model = DoubleBERT()
model.load_state_dict(torch.load('double_bert_sarcasm_detector.pt'))
model.to(device)
model.eval()

# Function to predict sarcasm
def predict_sarcasm(text, model, tokenizer, device, max_len=128):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, prediction = torch.max(outputs, dim=1)

    return prediction.item()

# Example usage
sample_text = "I just love getting stuck in traffic for hours!"
prediction = predict_sarcasm(sample_text, model, tokenizer, device)
label = "Sarcastic" if prediction == 1 else "Not Sarcastic"
print(f"Prediction: {label}")


In [ ]:
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def eval_model(model, data_loader, device, criterion):
    model = model.eval()
    val_loss = 0
    correct_predictions = 0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Get predictions
            _, preds = torch.max(outputs, dim=1)

            # Collect predictions and true labels
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

            correct_predictions += torch.sum(preds == labels)

    # Calculate accuracy
    val_accuracy = correct_predictions.double() / len(data_loader.dataset)

    return val_loss / len(data_loader), val_accuracy, val_preds, val_labels


In [ ]:
# Evaluate the model
val_loss, val_accuracy, val_preds, val_labels = eval_model(model, val_loader, device, criterion)


In [ ]:
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")


In [ ]:
from sklearn.metrics import classification_report
print("\nClassification Report:\n", classification_report(val_labels, val_preds))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Confusion Matrix
conf_matrix = confusion_matrix(val_labels, val_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.title("Validation Confusion Matrix")
plt.ylabel('Actual Labels')
plt.xlabel('Predicted Labels')
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='weighted')
print(f"Weighted Precision: {precision}")
print(f"Weighted Recall: {recall}")
print(f"Weighted F1 Score: {f1}")


In [ ]:
false_negatives = [val_texts[i] for i, (pred, true) in enumerate(zip(val_preds, val_labels)) if pred == 0 and true == 1]
false_positives = [val_texts[i] for i, (pred, true) in enumerate(zip(val_preds, val_labels)) if pred == 1 and true == 0]

print("Common False Negatives (sarcasm missed):")
for text in false_negatives[:5]:
    print(text)

print("\nCommon False Positives (non-sarcastic classified as sarcastic):")
for text in false_positives[:5]:
    print(text)


In [ ]:
from sklearn.metrics import f1_score

class_f1_scores = f1_score(val_labels, val_preds, average=None)
print(f"Class-wise F1-Scores: {class_f1_scores}")


In [ ]:
def eval_model(model, data_loader, device, criterion):
    model = model.eval()
    val_loss = 0
    correct_predictions = 0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Get predictions
            _, preds = torch.max(outputs, dim=1)

            # Collect predictions and true labels
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

            correct_predictions += torch.sum(preds == labels).item()  # Ensure it's a Python number

    # Calculate accuracy as a float
    val_accuracy = correct_predictions / len(data_loader.dataset)  # This is now a float

    return val_loss / len(data_loader), float(val_accuracy), val_preds, val_labels  # Ensure val_accuracy is a float


In [ ]:
val_loss, val_accuracy, val_preds, val_labels = eval_model(model, val_loader, device, criterion)

In [ ]:
# After your evaluation, you should have the following:
val_loss, val_accuracy, val_preds, val_labels = eval_model(model, val_loader, device, criterion)

# Define true_labels from your validation labels
true_labels = val_labels

# Convert predictions to probabilities (for binary classification)
# Assuming you have a binary classification with outputs as logits
val_preds_probs = [1 if pred == 1 else 0 for pred in val_preds]  # Modify based on your model's output

# Plotting ROC and Precision-Recall curves
plot_roc_curve(true_labels, val_preds_probs)
plot_precision_recall_curve(true_labels, val_preds_probs)


In [ ]:
# Verify lengths before plotting
print(f"Length of train_losses: {len(train_losses)}")
print(f"Length of val_losses: {len(val_losses)}")
print(f"Length of val_accuracies: {len(val_accuracies)}")


In [ ]:
# 2. ROC Curve
def plot_roc_curve(true_labels, predictions):
    fpr, tpr, _ = roc_curve(true_labels, predictions)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label='ROC Curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')  # Diagonal line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

# 3. Precision-Recall Curve
def plot_precision_recall_curve(true_labels, predictions):
    precision, recall, _ = precision_recall_curve(true_labels, predictions)
    plt.figure(figsize=(10, 6))
    plt.plot(recall, precision, color='green', lw=2)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.show()

# Assuming your final validation predictions are in val_preds and true_labels
# Convert val_preds to probabilities if necessary (e.g., for binary classification)
val_preds_probs = [1 if pred == 1 else 0 for pred in val_preds]  # Modify based on your model's output

# Plotting ROC and Precision-Recall curves
plot_roc_curve(true_labels, val_preds_probs)
plot_precision_recall_curve(true_labels, val_preds_probs)


In [ ]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix

# Extracting features and labels including attention mask
def extract_val_data(val_loader):
    X_val = []
    attention_masks = []
    y_val = []
    for data in val_loader:
        input_ids = data['input_ids']  # Replace with your actual input features
        attention_mask = data['attention_mask']
        labels = data['labels'].numpy()

        X_val.append(input_ids.cpu().numpy())  # Assuming input_ids is a tensor
        attention_masks.append(attention_mask.cpu().numpy())  # Assuming attention_mask is a tensor
        y_val.append(labels)

    return np.concatenate(X_val), np.concatenate(attention_masks), np.concatenate(y_val)

# Get validation data
X_val, attention_masks, y_val = extract_val_data(val_loader)

# Assuming you have already made predictions with your model
# y_pred = model.predict(X_val)  # Or however you obtain your predictions
# If you use PyTorch:
model.eval()
with torch.no_grad():
    inputs = torch.tensor(X_val).to(device)  # Move to device if needed
    masks = torch.tensor(attention_masks).to(device)  # Move attention masks to device

    # Pass both input_ids and attention_mask to the model
    outputs = model(input_ids=inputs, attention_mask=masks)
    _, y_pred = torch.max(outputs, 1)
    y_pred = y_pred.cpu().numpy()  # Convert to NumPy for further processing

# Continue with the rest of your code...


# 1. Permutation Importance
def plot_permutation_importance(model, X_val, y_val):
    result = permutation_importance(model, X_val, y_val, n_repeats=30, random_state=42, n_jobs=-1)
    sorted_idx = result.importances_mean.argsort()

    plt.figure(figsize=(10, 6))
    plt.barh(range(len(sorted_idx)), result.importances_mean[sorted_idx], yerr=result.importances_std[sorted_idx])
    plt.yticks(range(len(sorted_idx)), np.array(X_val.columns)[sorted_idx])  # Adjust for DataFrame or array
    plt.title("Permutation Importance")
    plt.xlabel("Importance Score")
    plt.show()

# Example call:
plot_permutation_importance(model, X_val, y_val)

# 2. Class Distribution
def plot_class_distribution(y):
    sns.countplot(x=y)
    plt.title("Class Distribution")
    plt.xlabel("Class")
    plt.ylabel("Count")
    plt.show()

# Example call:
plot_class_distribution(y_val)

# 3. Misclassification Plots
def plot_misclassified_samples(X_val, y_val, y_pred, n_samples=10):
    misclassified_idx = np.where(y_val != y_pred)[0]
    if len(misclassified_idx) == 0:
        print("No misclassifications found.")
        return

    # Select a few misclassified samples
    np.random.shuffle(misclassified_idx)
    misclassified_idx = misclassified_idx[:n_samples]

    plt.figure(figsize=(15, 5))
    for i, idx in enumerate(misclassified_idx):
        plt.subplot(2, n_samples // 2, i + 1)
        plt.imshow(X_val[idx], cmap='gray')  # Adjust based on your data format
        plt.title(f'True: {y_val[idx]}\nPredicted: {y_pred[idx]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Example call:
plot_misclassified_samples(X_val, y_val, y_pred)


In [ ]:
def plot_class_distribution(true_labels):
    plt.figure(figsize=(8, 5))
    sns.countplot(x=true_labels)
    plt.title('Class Distribution in Validation Set')
    plt.xlabel('Classes')
    plt.ylabel('Count')
    plt.show()

# Call the function
plot_class_distribution(true_labels)

# Analyzing Misclassifications
def analyze_misclassifications(true_labels, val_preds):
    misclassified = [(true, pred) for true, pred in zip(true_labels, val_preds) if true != pred]
    print("Misclassifications:")
    for true, pred in misclassified[:10]:  # Print the first 10 misclassifications
        print(f"True: {true}, Predicted: {pred}")

# Call the function
analyze_misclassifications(true_labels, val_preds)


In [ ]:
# 3. Misclassification Plots
def plot_misclassified_samples(X_val, y_val, y_pred, n_samples=10):
    misclassified_idx = np.where(y_val != y_pred)[0]
    if len(misclassified_idx) == 0:
        print("No misclassifications found.")
        return

    # Select a few misclassified samples
    np.random.shuffle(misclassified_idx)
    misclassified_idx = misclassified_idx[:n_samples]
    print(misclassified_idx)
    plt.figure(figsize=(15, 5))
    for i, idx in enumerate(misclassified_idx):
        plt.subplot(2, n_samples // 2, i + 1)
        plt.imshow(X_val[idx], cmap='gray')  # Adjust based on your data format
        plt.title(f'True: {y_val[idx]}\nPredicted: {y_pred[idx]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Example call:
plot_misclassified_samples(X_val, y_val, y_pred)